In [3]:
# Versi yang lebih ringkas, tanpa paket 'utils' yang tidak kita gunakan
!pip install flask pyngrok "langchain_community<0.3.0" replicate flask-cors --quiet

In [10]:
# SEL 2 (VERSI LENGKAP DENGAN AUTHTOKEN)

import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from google.colab import userdata

# Mengambil kunci API REPLICATE dari Colab Secrets
try:
    os.environ['REPLICATE_API_TOKEN'] = userdata.get('REPLICATE_API_TOKEN')
    print("✅ Kunci API Replicate berhasil diambil dari Secrets.")
except Exception as e:
    print(f"🛑 Gagal mengambil kunci API Replicate. Pastikan secret 'REPLICATE_API_TOKEN' sudah diatur.")

# Mengambil dan mengatur Authtoken NGROK dari Colab Secrets
try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("✅ Authtoken ngrok berhasil diatur dari Secrets.")
except Exception as e:
    print("⚠️ Authtoken ngrok tidak ditemukan di Secrets. Anda bisa menambahkannya untuk koneksi yang lebih stabil.")

✅ Kunci API Replicate berhasil diambil dari Secrets.
✅ Authtoken ngrok berhasil diatur dari Secrets.


In [5]:
# Sel 3: Inisialisasi Model AI
from langchain_community.llms import Replicate

try:
    model = Replicate(
        model="ibm-granite/granite-3.3-8b-instruct",
        replicate_api_token=os.environ.get('REPLICATE_API_TOKEN'),
        model_kwargs={"max_tokens": 2048, "temperature": 0.2, "stop_sequences": ["\n\n"]}
    )
    print("✅ Model AI berhasil diinisialisasi. Koneksi ke Replicate berhasil!")
except Exception as e:
    print(f"🛑 Gagal inisialisasi model: {e}")
    print("Error ini biasanya berarti REPLICATE_API_TOKEN Anda tidak valid atau salah.")

✅ Model AI berhasil diinisialisasi. Koneksi ke Replicate berhasil!


In [6]:
# Sel 4: Menyiapkan "Contekan" dan Fungsi Prompt

# !!! ISI BAGIAN INI DENGAN CONTOH-CONTOH ANDA !!!
# Pastikan formatnya adalah Python Dictionary
akcayaPhotoExamples = [
  {
    "question": "Buatkan komponen Header untuk website portofolio fotografi 'Akcaya Photo' dengan navigasi Home, Galeri, dan Kontak.",
    "context": "Header harus memiliki latar belakang gelap, logo di kiri, dan menu navigasi di kanan.",
    "output": """import React from 'react';

const Header = () => {
  return (
    <header className="bg-gray-900 text-white p-4 shadow-md">
      <div className="container mx-auto flex justify-between items-center">
        <h1 className="text-2xl font-bold tracking-wider">AKCAYA PHOTO</h1>
        <nav>
          <ul className="flex space-x-6">
            <li><a href="#" className="hover:text-blue-400 transition-colors">Home</a></li>
            <li><a href="#gallery" className="hover:text-blue-400 transition-colors">Galeri</a></li>
            <li><a href="#contact" className="hover:text-blue-400 transition-colors">Kontak</a></li>
          </ul>
        </nav>
      </div>
    </header>
  );
};

export default Header;"""
  },
  # Tambahkan contoh-contoh lain di sini...
  # { "question": "...", "context": "...", "output": """...""" },
]

def create_react_prompt(question, examples):
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )
    prompt = f"""You are an expert React developer specializing in Tailwind CSS. Your task is to generate a complete, copy-pasteable React functional component as a single JSX file. Here are some examples of your high-quality work:
    {formatted_examples}

    Now, based on the user's request, generate the component. Ensure the code is clean, well-formatted, and uses best practices.

    User Question: {question}
    Output only the JSX code for the component. Do not add any explanation or text besides the code."""
    return prompt

print("✅ Fungsi prompt dan data contoh berhasil dimuat.")

✅ Fungsi prompt dan data contoh berhasil dimuat.


In [7]:
# Sel 5: Membuat Web Server (API Endpoint)

app = Flask(__name__)
CORS(app) # Mengizinkan akses dari React App Anda

@app.route('/generate-code', methods=['POST'])
def generate_code_endpoint():
    try:
        data = request.get_json()
        if not data or 'question' not in data:
            return jsonify({"error": "Permintaan tidak valid, 'question' tidak ditemukan."}), 400

        user_question = data['question']

        prompt_text = create_react_prompt(user_question, akcayaPhotoExamples)
        generated_code = model.invoke(prompt_text)

        return jsonify({"generated_code": generated_code})

    except Exception as e:
        print(f"Error di dalam endpoint: {e}")
        return jsonify({"error": str(e)}), 500

print("✅ Logika server API berhasil didefinisikan.")

✅ Logika server API berhasil didefinisikan.


In [8]:
# Sel 6: Menjalankan Server dengan ngrok!

# Hentikan proses ngrok sebelumnya jika ada
ngrok.kill()

# Jalankan server di port 5000 dan buat URL publik
public_url = ngrok.connect(5000)
print("🚀 Server Anda sekarang LIVE dan bisa diakses di URL ini:")
print(public_url)
print("----------------------------------------------------")
print("Jangan tutup sel ini. Biarkan tetap berjalan untuk menjaga server tetap hidup.")

# Jalankan aplikasi Flask
app.run(port=5000)

🚀 Server Anda sekarang LIVE dan bisa diakses di URL ini:
NgrokTunnel: "https://01b7-34-106-222-229.ngrok-free.app" -> "http://localhost:5000"
----------------------------------------------------
Jangan tutup sel ini. Biarkan tetap berjalan untuk menjaga server tetap hidup.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Jun/2025 02:30:21] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2025 02:30:21] "GET /favicon.ico HTTP/1.1" 404 -
